<a href="https://colab.research.google.com/github/arthur-kt/PPP-Intertemporal-Converter/blob/main/PPP_converter_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PPP Interporal Converter

## Parâmetros Iniciais

In [1]:
!pip install fredapi
!pip install pycountry

from fredapi import Fred
import pycountry
import pandas as pd
from datetime import datetime, timedelta
import requests
from collections import defaultdict

# Inicialize a chave do FRED
fred = Fred(api_key='11cb1e53b57458acaddd29a16e0efe1b')


# Obtenha o CPI dos EUA para ajustar pela inflação
cpi_data = fred.get_series('CPIAUCSL')
cpi_df = cpi_data.to_frame(name='CPI')
cpi_df.index = pd.to_datetime(cpi_df.index)
cpi_df_ano = cpi_df.resample('YE').mean()  # Média anual do CPI

# CPI atual para trazer o valor a 2024
cpi_atual = cpi_df_ano.loc['2024'].values[0]

## Importar dados PPP

In [2]:
# Importar dados PPP
url = 'https://raw.githubusercontent.com/arthur-kt/PPP-Intertemporal-Converter/eabb3ca9caaca37dc9d9f1ae46ad03f7bc699954/ppp_att.csv'
ppp_data =  pd.read_csv(url, delimiter=',')

## Obter taxa de conversão do dólar 2024

In [3]:
# Função para obter o código de moeda usando o código de país ISO2
def get_currency_code(country_code):
    try:
        country = pycountry.countries.get(alpha_2=country_code)
        currency = pycountry.currencies.get(numeric=country.numeric)
        return currency.alpha_3 if currency else None
    except AttributeError:
        print(f"Não foi possível encontrar a moeda para o código de país: {country_code}")
        return None

# Mudar ISO3 para ISO2
def iso3_to_iso2(iso3):
    try:
    # Convert iso3 to string and ensure it's uppercase
        iso3 = str(iso3).upper()
        return pycountry.countries.get(alpha_3=iso3).alpha_2
    except (LookupError, AttributeError):
        # Handle both LookupError and AttributeError
        return None

# Importar taxa de câmbio dos países
url2 = 'https://raw.githubusercontent.com/arthur-kt/PPP-Intertemporal-Converter/5b0b2f6071f19198b91a8ef10e2cfef3f3a1b91c/exchange_rates.csv'
exchange_rates24 =  pd.read_csv(url2)
exchange_rates24 = exchange_rates24[['Country Code', '2024 [2024]']]
exchange_rates24 = exchange_rates24.rename(columns={'2024 [2024]': '2024'})
exchange_rates24['Country Code'] = exchange_rates24['Country Code'].apply(iso3_to_iso2)

## Função Conversão

In [19]:
# Função para obter o PPP ajustado de um país em um ano específico a partir do CSV
def get_ppp_adjusted(country_code, year):
    ppp_row = ppp_data[(ppp_data['Country ID'] == country_code) & (ppp_data['Year'] == year)]
    return ppp_row.iloc[0]['PPP']

# Função para converter gasto local em dólar ajustado pela inflação dos EUA
def converter_para_dolar_corrente(row):
    # Para o ano 2023, usa os dados de PPP externos
    if row['Ano2'] == 0:
      if row['Ano'] == 2024:
        # Procura o exchange rate diretamente no dataframe exchange_rates24
            exchange_rate_row = exchange_rates24[exchange_rates24['Country Code'] == row['País_Moeda']]
            if not exchange_rate_row.empty:
                exchange_rate = exchange_rate_row.iloc[0]['2024']  # Obtém a taxa de câmbio para 2024
                exchange_rate = float(exchange_rate)
                gasto_em_dolar_2024 = row['Gasto_Local'] / exchange_rate
                return gasto_em_dolar_2024
      else:
          # Para demais anos
           ppp_adjusted = get_ppp_adjusted(row['País_Moeda'], row['Ano'])
           gasto_em_dolar_ppp = row['Gasto_Local'] / ppp_adjusted

      # Ajuste pela inflação dos EUA
      cpi_ano_referencia = cpi_df_ano.loc[str(row['Ano'])].values[0]
      gasto_ajustado = gasto_em_dolar_ppp * (cpi_atual / cpi_ano_referencia)

    else:
      # Caso exista Ano2, calcula o PPP médio entre Ano1 e Ano2

      ppp_ano1 = get_ppp_adjusted(row['País_Moeda'], row['Ano'])
      ppp_ano2 = get_ppp_adjusted(row['País_Moeda'], row['Ano2'])

      # Média dos gastos enre entre Ano1 e Ano2
      gasto_em_dolar_ppp = row['Gasto_Local'] / ((ppp_ano1 + ppp_ano2) / 2)


      # Ajuste pela inflação dos EUA usando CPI de Ano1 e Ano2

      # Obter o índice CPI de ambos os anos
      cpi_ano1 = cpi_df_ano.loc[str(row['Ano'])].values[0]
      cpi_ano2 = cpi_df_ano.loc[str(row['Ano2'])].values[0]

      # Calcula a média do CPI para ajuste pela inflação
      cpi_media = (cpi_ano1 + cpi_ano2) / 2

      # Ajusta o gasto usando a média do CPI
      gasto_ajustado = gasto_em_dolar_ppp * (cpi_atual / cpi_media)

    return float(gasto_ajustado)


def agrupar_gastos(df):
    # Agrupa por 'Período' e 'País do Programa' e soma os valores de 'Gasto_Convertido_Dolar_2024'
    df_agrupado = df.groupby(['Período', 'País do Programa'])['Gasto_Convertido_Dolar_2024'].sum().reset_index()
    return df_agrupado

def agrupar_gastos_pais(df):
    # Agrupa por 'Período' e 'País do Programa' e soma os valores de 'Gasto_Convertido_Dolar_2024'
    df_pais = df.groupby(['País do Programa'])['Gasto_Convertido_Dolar_2024'].sum().reset_index()
    return df_pais

def agrupar_gastos_programa(df):
    # Agrupa por 'Período' e 'País do Programa' e soma os valores de 'Gasto_Convertido_Dolar_2024'
    df_prog = df.groupby(['Programa', 'País do Programa'])['Gasto_Convertido_Dolar_2024'].sum().reset_index()
    return df_prog

## Função Formatação

In [5]:
# Função para formatar valores em milhões, bilhões ou trilhões para Moeda Local
def formatar_valor_moeda_local(valor, codigo_pais):
    currency_code = get_currency_code(codigo_pais)  # Obtém o código da moeda do país
    if not currency_code:
        currency_code = "Moeda desconhecida"  # Se não houver código, exibe "Moeda desconhecida"

    if valor >= 1_000_000_000_000:  # Trilhão
        return f"{currency_code} {valor / 1_000_000_000_000:.2f} trilhões"
    elif valor >= 1_000_000_000:  # Bilhão
        return f"{currency_code} {valor / 1_000_000_000:.2f} bilhões"
    elif valor >= 1_000_000:  # Milhão
        return f"{currency_code} {valor / 1_000_000:.2f} milhões"
    else:
        return f"{currency_code} {valor:.2f}"

# Função para formatar valores em milhões, bilhões ou trilhões para Dólar Ajustado
def formatar_valor_dolar(valor):
    if valor >= 1_000_000_000_000:  # Trilhão
        return f"USD {valor / 1_000_000_000_000:.2f} trilhões"
    elif valor >= 1_000_000_000:  # Bilhão
        return f"USD {valor / 1_000_000_000:.2f} bilhões"
    elif valor >= 1_000_000:  # Milhão
        return f"USD {valor / 1_000_000:.2f} milhões"
    else:
        return f"USD {valor:.2f}"

# Função para juntar Ano com Ano2
def definir_periodo(df):
    # Verifica se as colunas Ano e Ano2 existem no DataFrame
    if 'Ano' in df.columns and 'Ano2' in df.columns:
        # Aplica a condição para criar a coluna Período
        df['Período'] = df.apply(lambda row: str(row['Ano']) if row['Ano2'] == 0 else f"{row['Ano']} à {row['Ano2']}", axis=1)
    else:
        print("Colunas 'Ano' e 'Ano2' não encontradas no DataFrame.")
    return df

## Insira seus dados:

In [6]:
# Dados

## Insira em 'País_Moeda' o o país emissor da moeda utilizada (no caso de uso de moedas diferentes do país de origem), em 'País do Programa' o país de origem do Programa, em 'Ano' o ano da origem dos recursos, em 'Gasto_Local' a quantia dispendida, em 'Programa' o nome do Programa e em 'Descrição' uma breve descrição da função ou destinação dos valores.
## A coluna 'Ano2' serve para os programas que tem custos entre 2 diferentes anos, sendo necessário calcular a média do PPP e da inflação americana. Então, insira no 'Ano2' o último ano do programa, se não, preencha com 0.

dados = {
    'País_Moeda': ['US', #1
                   'US', #2
                   'US', #3
                   'US', #4
                   'US', #5
                   'US', #6
                   'US', #7
                   'US', #8
                   'US', #9
                   'US', #10
                   'US', #11
                   'US', #12
                   'US', #13
                   'US', #14
                   'US', #15
                   'US', #16
                   'US', #17
                   'GB', #18
                   'GB', #19
                   'GB', #20
                   'GB', #21
                   'GB', #22
                   'GB', #23
                   'GB', #24
                   'GB', #25
                   'GB', #26
                   'GB', #27
                   'GB', #28
                   'KR', #29
                   'KR', #30
                   'KR', #31
                   'KR', #32
                   'KR', #33
                   'KR', #34
                   'KR', #35
                   'KR', #36
                   'KR', #37
                   'KR', #38
                   'KR', #39
                   'KR', #40
                   'KR', #41
                   'KR', #42
                   'KR', #43
                   'KR', #44
                   'KR', #45
                   'KR', #46
                   'KR', #47
                   'KR', #48
                   'IN', #49
                   'IN', #50
                   'IN', #51
                   'IN', #52
                   'IN', #53
                   'IN', #54
                   'IN', #55
                   'IN', #56
                   'IN', #57
                   'IN', #58
                   'IN', #59
                   'ZA', #60
                   'ZA', #61
                   'ZA', #62
                   'ZA', #63
                   'ZA', #64
                   'ZA', #65
                   'US',  #1
             'US',  #2
             'US',  #3
             'US',  #4
             'US',  #5
             'KR',  #6
             'KR',  #7
             'KR',  #8
             'KR',  #9
             'KR',  #10
             'GB',  #11
             'GB',  #12
             'GB',  #13
             'GB',  #14
             'GB',  #15
             'GB',  #16
             'US',  #17
             'GB',  #18
             'GB',  #19
             'GB',  #20
             'GB',  #21
             'CN', #22
             'US', #23
             'CN', #24
             'CN', #25
             'CN', #26
             'CN', #27
             'US', #28
             'CN', #29
             'IN', #30
             'IN', #31
             'IN', #32
             'IN', #33
             'IN', #34
             'ZA', #35
             'ZA', #36
             'ZA', #37
             'MX', #38
             'MX', #39
             'MX', #40
             'MX', #41
             'MX', #42
             'MX', #43
             'MX', #44
             'MX',  #45
             'IN', #46
             'KR', #47
             'IN', #48
             'IN', #49
             'GB', #50
             'GB', #51
             'IN', #52
             ],  # Usando códigos ISO2 dos países

        'País do Programa': ['US', #1
                             'US', #2
                             'US', #3
                             'US', #4
                             'US', #5
                             'US', #6
                             'US', #7
                             'US', #8
                             'US', #9
                             'US', #10
                             'US', #11
                             'US', #12
                             'US', #13
                             'US', #14
                             'US', #15
                             'US', #16
                             'US', #17
                             'GB', #18
                             'GB', #19
                             'GB', #20
                             'GB', #21
                             'GB', #22
                             'GB', #23
                             'GB', #24
                             'GB', #25
                             'GB', #26
                             'GB', #27
                             'GB', #28
                             'KR', #29
                             'KR', #30
                             'KR', #31
                             'KR', #32
                             'KR', #33
                             'KR', #34
                             'KR', #35
                             'KR', #36
                             'KR', #37
                             'KR', #38
                             'KR', #39
                             'KR', #40
                             'KR', #41
                             'KR', #42
                             'KR', #43
                             'KR', #44
                             'KR', #45
                             'KR', #46
                             'KR', #47
                             'KR', #48
                             'IN', #49
                             'IN', #50
                             'IN', #51
                             'IN', #52
                             'IN', #53
                             'IN', #54
                             'IN', #55
                             'IN', #56
                             'IN', #57
                             'IN', #58
                             'IN', #59
                             'ZA', #60
                             'ZA', #61
                             'ZA', #62
                             'ZA', #63
                             'ZA', #64
                             'ZA', #65
                             'US',  #1
             'US',  #2
             'US',  #3
             'US',  #4
             'US',  #5
             'KR',  #6
             'KR',  #7
             'KR',  #8
             'KR',  #9
             'KR',  #10
             'GB',  #11
             'GB',  #12
             'GB',  #13
             'GB',  #14
             'GB',  #15
             'GB',  #16
             'GB',  #17
             'GB',  #18
             'GB',  #19
             'GB',  #20
             'GB',  #21
             'CN', #22
             'CN', #23
             'CN', #24
             'CN', #25
             'CN', #26
             'CN', #27
             'CN', #28
             'CN', #29
             'IN', #30
             'IN', #31
             'IN', #32
             'IN', #33
             'IN', #34
             'ZA', #35
             'ZA', #36
             'ZA', #37
             'MX', #38
             'MX', #39
             'MX', #40
             'MX', #41
             'MX', #42
             'MX', #43
             'MX', #44
             'MX', #45
             'IN', #46
             'KR', #47
             'IN', #48
             'IN', #49
             'GB', #50
             'GB', #51
             'IN', #52
             ],  # Usando códigos ISO2 dos países

    'Ano': [2020, #1
            2020, #2
            2020, #3
            2020, #4
            2024, #5
            2021, #6
            2022, #7
            2023, #8
            2024, #9
            2024, #10
            2022, #11
            2022, #12
            2019, #13
            2020, #14
            2021, #15
            2021, #16
            2021, #17
            2018, #18
            2018, #19
            2018, #20
            2018, #21
            2018, #22
            2021, #23
            2021, #24
            2021, #25
            2023, #26
            2023, #27
            2024, #28
            2020, #29
            2024, #30
            2014, #31
            2022, #32
            2014, #33
            2023, #34
            2017, #35
            2019, #36
            2019, #37
            2020, #38
            2020, #39
            2023, #40
            2023, #41
            2023, #42
            2023, #43
            2023, #44
            2023, #45
            2023, #46
            2023, #47
            2023, #48
            2020, #49
            2023, #50
            2020, #51
            2020, #52
            2020, #53
            2024, #54
            2020, #55
            2021, #56
            2021, #57
            2014, #58
            2019, #59
            2024, #60
            2020, #61
            2019, #62
            2019, #63
            2022, #64
            2019, #65
            2021,  #1
            2021,  #2
            2021,  #3
            2023,  #4
            2022,  #5
            2023,  #6
            2023,  #7
            2023,  #8
            2023,  #9
            2023,  #10
            2023,  #11
            2023,  #12
            2024,  #13
            2017,  #14
            2017,  #15
            2019,  #16
            2017,  #17
            2019,  #18
            2021,  #19
            2022,  #20
            2022,  #21
            2016, #22
            2019, #23
            2009, #24
            2019, #25
            2009, #26
            2020, #27
            2008, #28
            2021, #29
            2012, #30
            2018, #31
            2023, #32
            2020, #33
            2021, #34
            2016, #35
            2012, #36
            2011, #37
            2008, #38
            2007, #39
            2006, #40
            2009, #41
            2013, #42
            2013, #43
            2021, #44
            2021, #45
            2018, #46
            2024, #47
            2021, #48
            2020, #49
            2021, #50
            2021, #51
            2021, #52
            ],

   'Ano2': [0, #1
            0, #2
            0, #3
            0, #4
            0, #5
            0, #6
            0, #7
            0, #8
            0, #9
            0, #10
            0, #11
            0, #12
            0, #13
            0, #14
            0, #15
            0, #16
            0, #17
            0, #18
            0, #19
            0, #20
            0, #21
            0, #22
            0, #23
            0, #24
            0, #25
            0, #26
            0, #27
            0, #28
            0, #29
            0, #30
            0, #31
            0, #32
            2022, #33
            0, #34
            0, #35
            0, #36
            0, #37
            0, #38
            0, #39
            0, #40
            0, #41
            0, #42
            0, #43
            0, #44
            0, #45
            0, #46
            0, #47
            0, #48
            0, #49
            0, #50
            2023, #51
            0, #52
            0, #53
            0, #54
            0, #55
            0, #56
            0, #57
            0, #58
            0, #59
            0, #60
            0, #61
            2023, #62
            2023, #63
            0, #64
            2023, #65
            0,  #1
            0,  #2
            0,  #3
            0,  #4
            0,  #5
            0,  #6
            0,  #7
            0,  #8
            0,  #9
            0,  #10
            0,  #11
            0,  #12
            0,  #13
            0,  #14
            0,  #15
            0,  #16
            0,  #17
            0,  #18
            0,  #19
            0,  #20
            0,  #21
            0, #22
            0, #23
            2016, #24
            0, #25
            0, #26
            0, #27
            2020, #28
            0, #29
            2017, #30
            2020, #31
            0, #32
            0, #33
            0, #34
            0, #35
            0, #36
            2015, #37
            0, #38
            0, #39
            0, #40
            0, #41
            2018, #42
            2018, #43
            0, #44
            0, #45
            2020, #46
            0, #47
            0, #48
            0, #49
            0, #50
            0, #51
            0, #52
            ],

    'Gasto_Local': [2900000000, #1
                    208000000, #2
                    8400000000, #3
                    1513000000000, #4
                    31100000000, #5
                    382200000, #6
                    953200000, #7
                    1193200000, #8
                    993200000, #9
                    1195200000, #10
                    156800000, #11
                    88000000000, #12
                    7500000000, #13
                    5000000, #14
                    65300000000, #15
                    18230000000, #16
                    951800000, #17
                    21000000, #18
                    181000000, #19
                    66000000, #20
                    40000000, #21
                    280000000, #22
                    140000000, #23
                    200000000, #24
                    370000000, #25
                    4500000000, #26
                    520000000, #27
                    520000000, #28
                    849200000000, #29
                    471500000000, #30
                    26800000000, #31
                    293400000000, #32
                    1144900000000, #33
                    31500000000, #34
                    2600000000000, #35
                    4000000000000, #36
                    2000000000000, #37
                    2800000000000, #38
                    595500000000, #39
                    500000000000, #40
                    261600000000, #41
                    2200000000000, #42
                    1200000000000, #43
                    2950000000, #44
                    46650000000, #45
                    279700000000, #46
                    361300000000000, #47
                    1000000000000, #48
                    993115200000, #49
                    1128994700000, #50
                    454880000000, #51
                    6300000000, #52
                    69400000000, #53
                    40240000000, #54
                    30000000000, #55
                    54000000000, #56
                    8739300000, #57
                    1250000000, #58
                    29000000000, #59
                    8000000, #60
                    3175677, #61
                    1500000000000, #62
                    100000000000, #63
                    7700000000, #64
                    10000000000, #65
                    4460000000,  #1
                    2650000000,  #2
                    6500000000,  #3
                    2500000000,  #4
                    2105000000,  #5
                    780100000000,  #6
                    49500000000,  #7
                    1880000000000,  #8
                    60400000000,  #9
                    2100000000000,  #10
                    1100000000,  #11
                    17000000,  #12
                    7600000,  #13
                    5000000000,  #14
                    80000000000,  #15
                    4000000000,  #16
                    3000000000,  #17
                    948100000000,  #18
                    560000000,  #19
                    650000000,  #20
                    520000000,  #21
                    16000000000000, #22
                    291000000000, #23
                    5640000000000, #24
                    1861695000000, #25
                    15, #26
                    74, #27
                    3800000000, #28
                    1147000000, #29
                    1250000000, #30
                    4000000000, #31
                    3000000000, #32
                    69400000000, #33
                    5000000000, #34
                    14426000000, #35
                    1600000000, #36
                    13747152, #37
                    4500000000, #38
                    4500000000, #39
                    4000000000, #40
                    1700000000, #41
                    2138280000, #42
                    11900000, #43
                    89139000000, #44
                    44048700000, #45
                    1440000000, #46
                    55000000000, #47
                    150000000000, #48
                    34200000000, #49
                    14600000000, #50
                    800000000, #51
                    50000000000, #52

                    ],  # Gasto em moeda local

    'Programa': ['Defense Production Act (DPA)', #1
                 'Defense Production Act (DPA)', #2
                 'Defense Production Act (DPA)', #3
                 'Defense Production Act (DPA)', #4
                 'Global Health Security', #5
                 'Global Health Security', #6
                 'Global Health Security', #7
                 'Global Health Security', #8
                 'Global Health Security', #9
                 'Global Health Security', #10
                 'NATIONAL BIODEFENSE STRATEGY AND IMPLEMENTATION', #11
                 'NATIONAL BIODEFENSE STRATEGY AND IMPLEMENTATION', #12
                 'National Biodefense Strategy', #13
                 'National Biodefense Strategy', #14
                 'American Pandemic Preparedness: Transforming Our Capabilities', #15
                 'Operação Warp Speed (OWS)', #16
                 'Operação Warp Speed (OWS)', #17
                 'Industrial Strategy Challenge Fund: Leading-edge healthcare', #18
                 'Industrial Strategy Challenge Fund: Leading-edge healthcare', #19
                 'Industrial Strategy Challenge Fund: Leading-edge healthcare', #20
                 'Industrial Strategy Challenge Fund: Healthy ageing', #21
                 'Industrial Strategy Challenge Fund: Healthy ageing', #22
                 'Life Sciences Vision', #23
                 'Life Sciences Vision', #24
                 'Life Sciences Vision', #25
                 'Advanced Manufacturing Plan', #26
                 'Advanced Manufacturing Plan', #27
                 'Life Sciences Innovative Manufacturing Fund', #28
                 'MSS - Ministry of SMEs and Startups', #29
                 'Tech Incubation Program for Start-ups (TIPS)', #30
                 'Tech Incubation Program for Start-ups (TIPS)', #31
                 'Tech Incubation Program for Start-ups (TIPS)', #32
                 'Tech Incubation Program for Start-ups (TIPS)', #33
                 'Deeptech Tech Incubator Program for Startup', #34
                 'Bio-Health Industry Innovation Strategy', #35
                 'Bio-Health Industry Innovation Strategy', #36
                 'Bio-Health Industry Innovation Strategy', #37
                 'Bio-Health Industry Innovation Strategy', #38
                 'Bio-Health Industry Innovation Strategy', #39
                 'K-Bio Vaccine Fund', #40
                 'K-Bio Vaccine Fund', #41
                 'Strategies for Creating New Bio-Health Markets', #42
                 'Strategies for Creating New Bio-Health Markets', #43
                 'Strategies for Creating New Bio-Health Markets', #44
                 'Strategies for Creating New Bio-Health Markets', #45
                 'Strategies for Creating New Bio-Health Markets', #46
                 'Strategies for Creating New Bio-Health Markets', #47
                 'Comprehensive Plan to Promote the Biopharma Industry', #48
                 'National Education Policy', #49
                 'National Education Policy', #50
                 'National Education Policy', #51
                 'National Education Policy', #52
                 'Production Linked Incentive (PLI) Scheme for promotion of domestic manufacturing of critical Key Starting Materials (KSMs)/ Drug Intermediates(DIs) and Active Pharmaceutical Ingredients (APIs)', #53
                 'Production Linked Incentive (PLI) Scheme for promotion of domestic manufacturing of critical Key Starting Materials (KSMs)/ Drug Intermediates(DIs) and Active Pharmaceutical Ingredients (APIs)', #54
                 'Scheme for Promotion of Bulk Drug Parks', #55
                 'Production Linked Incentive Scheme (PLI) for Pharmaceuticals', #56
                 'Production Linked Incentive Scheme (PLI) for Pharmaceuticals', #57
                 'Cluster Development Programme for Pharma Sector (CDP-PS)', #58
                 'Credit Linked Capital Subsidy (CLCS)', #59
                 'Technology and Human Resources for Industry Programme (THRIP)', #60
                 'Programa de Suporte para Inovação Industrial (SPII)', #61
                 'Reimagined Industrial Strategy', #62
                 'Reimagined Industrial Strategy', #63
                 'Reimagined Industrial Strategy', #64
                 'Reimagined Industrial Strategy', #65
                 'EOs 14017 and 14001',  #1
                 'EOs 14017 and 14001',  #2
                 'ARPA-H',  #3
                 'ARP A-H',  #4
                 'Executive Order on Advancing Biotechnology and Biomanufacturing Innovation',  #5
                 'Orçamento 2024',  #6
                 'Korean ARPA-H',  #7
                 'Korean ARPA-H',  #8
                 'Boston-Korea',  #9
                 'First National Strategic Technology Nurture Plan',  #10
                 'Medicines Manufacturing Industry Partnership (MMIP)',  #11
                 'Life Sciences Innovative Manufacturing Fund (LSIMF)',  #12
                 'Life Sciences Innovative Manufacturing Fund (LSIMF)',  #13
                 'The Life Sciences Industrial Strategy',  #14
                 'The Life Sciences Industrial Strategy',  #15
                 'The Life Sciences Industrial Strategy',  #16
                 'The Life Sciences Industrial Strategy',  #17
                 'The Life Sciences Industrial Strategy',  #18
                 'UK Innovation Strategy',  #19
                 'Life Sciences 2030 Skills Strategy',  #20
                 'Life Sciences 2030 Skills Strategy',  #21
                 'Healthy China 2030', #22
                 'Healthy China 2030', #23
                 'Healthy China 2030', #24
                 '14º Plano Quinquenal Chinês (2021-2025)', #25
                 '14º Plano Quinquenal Chinês (2021-2025)', #26
                 '14º Plano Quinquenal Chinês (2021-2025)', #27
                 '14º Plano Quinquenal para o Desenvolvimento da Bioeconomia', #28
                 'Relatório de autoavaliação sobre o desempenho geral dos fundos de subsídios regulatórios de medicamentos em 2021', #29
                 'Programa de Desenvolvimento de Clusters para o Setor Farmacêutico (CDP-PS)', #30
                 'Programa de Desenvolvimento de Clusters para o Setor Farmacêutico (CDP-PS)', #31
                 'Programa de Desenvolvimento de Clusters para o Setor Farmacêutico (CDP-PS)', #32
                 'Programas de incentivos vinculados (PLI)', #33
                 'Strengthening of Pharmaceutical Industry Scheme', #34
                 'Investimento em P&D GOV 2016', #35
                 'Industrial Policy Action Plan - Ketlaphela', #36
                 'Industrial Policy Action Plan - Ketlaphela', #37
                 'Estímulos Fiscales', #38
                 'Estímulos Fiscales', #39
                 'Estímulos Fiscales', #40
                 'Programa de Estímulos à Inovação (PEI)', #41
                 'Programa de Estímulos à Inovação (PEI', #42
                 'Programa de Estímulos à Inovação (PEI)', #43
                 'Programa Especial de Ciencia, Tecnología e Innovación 2021-2024', #44
                 'Programa Especial de Ciencia, Tecnología e Innovación 2021-2024',  #45
                 'Programa de assistência para atualização de tecnologia farmacêutica', #46
                 'Korean ARPA-H',  #47
                 'Programas de incentivos vinculados (PLI)', #48
                 'Programas de incentivos vinculados (PLI)', #49
                 'Build Back Better: Our Plan for Growth and Innovation', #50
                 'Build Back Better: Our Plan for Growth and Innovation', #51
                 'Strengthening of Pharmaceutical Industry Scheme', #52
                 ],  # Nomes dos programas

    'Descrição': ['Title I: Used to purchase over 187,000 ventilators by the end of the year', #1
                  'Title III: Increase capacity for nasal swabs and respirators in limited amounts', #2
                  'Obrigou este valor à compra de fornecimentos e à reposição do Estoque Nacional Estratégico', #3
                  'Award for domestic production expansion projects for medical supplies using CARES Act appropriations', #4
                  'Experts estimate this additional amount annually, of which two-thirds would come from domestic financing and one-third from external financing, is needed to strengthen outbreak and pandemic prevention, preparedness, and response capacities.', #5
                  'USAID/State and CDC GHS Funding - FY 2021 Enacted', #6
                  'USAID/State and CDC GHS Funding - FY 2022 Enacted', #7
                  'USAID/State and CDC GHS Funding - FY 2023 Enacted', #8
                  'USAID/State and CDC GHS Funding - FY 2024 Enacted', #9
                  'USAID/State and CDC GHS Funding - FY 2025 President Budget', #10
                  'Beyond the Needle program e Coalition for Epidemic Preparedness Innovations', #11
                  'President’s request over five years for pandemic preparedness and biodefense', #12
                  'Reported spending on biodefense across all D / As for 2019', #13
                  'Orçamento pedido pelo HSS para implementar a National Biodefense Strategy. No entanto, o orçamento de 2020 não incluíu esse valor requisitado.', #14
                  'Custo estimado para período entre 7 e 10 anos', #15
                  'Valor dos contratos de Vaccine Candidates Supported by BARDA and Other Federal Agencies', #16
                  'Valor dos Federal Government Contracts for Ancillary COVID-19 Vaccine Supplies', #17
                  'Financiamento para 3 centros de tratamento de terapias avançadas', #18
                  'Desafio para desenvolvimento de novas medicinas, vacinas e outros produtos da saúde, a ser investido em 4 anos', #19
                  'Investimento no Vaccines Manufacturing Innovation Centre', #20
                  'Investimento no UK Dementia Research Institute (UKDRI)', #21
                  'Investimento em centros regionais de exelência e um programa de uso de dados para medicina de precisão e diagnósticos preventivos', #22
                  'NHS AI Health and Care Award delivered by the NHSX AI Lab in partnership with the AAC', #23
                  'Lançamento do Life Sciences Investment Programme (LSIP)', #24
                  'Aumento da capacidade de manufatura de vacinas e fundo de apoio a manufatura', #25
                  'Montante total disponibilizado para financiamento no âmbito do plano a partir de 2025, por cinco anos (2025-2030)', #26
                  'Parte do plano destinada às indústrias da ciências da vida', #27
                  'Segunda Fase (Valor atualizado out/24)', #28
                  'Orçamento para o ministério em 2020', #29
                  'Suporte planejado para 1925 empresas', #30
                  'Orçamento TIPS em 2014', #31
                  'Orçamento TIPS em 2022', #32
                  'Alocação governamental acumulada entre 2014 e 2022', #33
                  'Anunciado que o programa suportaria 120 projetos com este orçamento', #34
                  'Antes da estratégia: investimentos em P&D na biotecnologia e no setor da saúde em 2017', #35
                  'Meta de investimentos em P&D na biotecnologia e no setor da saúde para 2025', #36
                  'Policy loans a year for the next five years', #37
                  'O governo planeia investir no desenvolvimento de dois projectos interagências – desenvolvimento de novos medicamentos e desenvolvimento de tecnologia de medicina regenerativa. Os dois projetos nacionais fazem parte da Bio-Health Industry Innovation Strategy', #38
                  'Investimento na área de medicina regenerativa por 10 anos a partir de 2021', #39
                  'A futura tecnologia médica será financiada através da criação do K-Bio Vaccine Fund, no primeiro semestre de 2023, para expandir o investimento no desenvolvimento de vacinas e terapêuticos.', #40
                  'K-Bio Vaccine Fund Nos. 1 e 2 arrecadaram este valor em investimentos.', #41
                  'The National New Drug Development Project will continue to be invested between 2021 and 2030', #42
                  'Stage 1 of the Interdepartmental Full-Cycle Medical Device R&D project will be executed between 2020 and 2025', #43
                  'Expansão do Turismo Médico Internacional e Saúde Digital', #44
                  'Fortalecimento de P&D e Inovação em Bio-saúde', #45
                  'Desenvolvimento de Infraestrutura e Apoio a Startups Bio-saúde', #46
                  'Criação de Estruturas Jurídicas e Infraestrutura para Exportação: ampliação do financiaento para Seguro de Exportação, Banco de Exportação-Importação da Coreia, Fundo de Garantia de Crédito da Coreia, Financiamento de Tecnologia da Coreia e Banco de Desenvolvimento da Coreia e Banco Industrial da Coreia', #47
                  'Will be invested in the K-Bio Vaccine Fund, and based on the analysis of the results, another mega fund will be set up', #48
                  'Orçamento anual do Ministério da Educação em 2020-21', #49
                  'Orçamento anual do Ministério da Educação em 2023-24', #50
                  'Orçamento Total do programa PM SHRI para Modernização de Escolas ao longo de 5 anos, dos quais 39.85% dos valores serão fornecidos pelo governo central', #51
                  'Investimento Inicial', #52
                  'O período de produção sob o esquema é do ano fiscal de 2022-23 ao ano fiscal de 2028-29.', #53
                  'Investimento Alcançado em 2024. Projetos concluídos têm uma capacidade combinada de produção de 56.679 toneladas por ano.', #54
                  'Orçamento de 2020-21 até 2024-25. Busca estabelecer três parques de produção de ingredientes farmacêuticos ativos (IFAs) e intermediários de medicamentos com infraestrutura comum avançada.', #55
                  'Critical Key Starting materials/Drug Intermediaries and Active Pharmaceutical Ingredients – 47 applications approved', #56
                  'Manufacturing of Medical Devices – 14 applications are approved', #57
                  'Proposed as a Central Sector Scheme for the remaining years of the 12th Plan and also to continue in the next Five Year Plan', #58
                  'Desembolso', #59
                  'Gasto compartilhado máximo por ano por qualquer número de projetos', #60
                  'Total programme and operating expenses of 2020', #61
                  'Investment pledges in 317 projects', #62
                  'Industrial funding provided over five years', #63
                  'Opening of the new Sappi-Saiccor facility, works by processing raw timber into pulp that can be used for viscose, pharmaceutical products and other industrial applications.', #64
                  'The IDC facilitated localisation and beneficiation activities across sectors such as steel, pharmaceuticals, automotive and food products',#65
                  'Valores gasto',  #1
                  'Investimentos planejados',  #2
                  '2022-2024',  #3
                  '2024-2025',  #4
                  'Objective to improve and expand domestic biomanufacturing production capacity and processes',  #5
                  'Feito em 2023',  #6
                  'Alocado',  #7
                  'Entre 2024-2033',  #8
                  'Sem descrição',  #9
                  'De 2023 a 2028',  #10
                  'Follow the Green, High-Tech Road',  #11
                  'Primeira fase',  #12
                  'Segunda fase (2024)',  #13
                  '-',  #14
                  'Over the next 10 years',  #15
                  'Atingir esse nível de gasto governamental em P&D em 2027',  #16
                  'Gasto governamental em P&D em 2017',  #17
                  '-',  #18
                  '-',  #19
                  '-',  #20
                  'Entre 2025-2030',  #21
                  'De 2016 a 2030', #22
                  'Government R&D spending on biotech', #23
                  'From 2009 - 2016 to improve its healthcare system', #24
                  'Valores não relacionados ao plano: Despesas governamentais com saúde', #25
                  'Valores não relacionados ao plano: Subsídios governamentais na saúde por pessoa 2009', #26
                  'Valores não relacionados ao plano: Subsídios governamentais na saúde por pessoa 2020', #27
                  'Valores não relacionados ao plano: Investing over the period of 2008–2020 in biotechnology R&D', #28
                  'Fundos de subsídio para supervisão de alimentos e medicamentos alocados pelo governo central em 2021: valor para produtos farmacêuticos', #29
                  'Período de 2012 a 2017', #30
                  'Período de 2018 a 2020 (50% Promoção de Parques de IFAs + 50% assistência para parques de dispositivos médicos)', #31
                  'Orçamento ampliado em 2023 até o ano fiscal de 2025-2026', #32
                  'Incentivo para produção doméstica de materiais de partida, intermediários de síntese e IFAs', #33
                  'Orçamento para o período de 2021-2026', #34
                  'Gastos com P&D em 2016 (Geral, não apenas saúde)', #35
                  'Investimento total projetado em 2012', #36
                  'Financiamento de fato de 2011 a 2015', #37
                  'Programa de Estímulos Fiscales al Gasto en Investigación y Desarrollo de Tecnología de las Empresas Privadas en México', #38
                  'Programa de Estímulos Fiscales al Gasto en Investigación y Desarrollo de Tecnología de las Empresas Privadas en México', #39
                  'Programa de Estímulos Fiscales al Gasto en Investigación y Desarrollo de Tecnología de las Empresas Privadas en México', #40
                  '-', #41
                  'Destinação ao setor Farmacêutico de 2013-2018  (proxy com % de projetos do ramo farmacêutico)', #42
                  'Investigação em biotecnologia para fins não agrícolas de 2013 a 2018', #43
                  'Gasto federal en ciencia y tecnología en temas prioritarios (um deles é Saúde)', #44
                  'Gasto federal en investigación Científica y desarrollo experimental en temas prioritarios (um deles é Saúde)',  #45
                  'Período de 2018 a 2020', #46
                  'Research funding over the next 5 years', #47
                  'Programa para medicamentos', #48
                  'Programa para dispositivos médicos', #49
                  'Não específico para indústria da saúde: government investment in R&D in 2021-22', #50
                  'Não específico para indústria da saúde: funding for the Advanced Research & Invention Agency (ARIA)', #51
                  'O governo indiano planeja gastar no programa National Policy on R&D and Innovation in the PharmaMedTech Sector in India', #52
                  ]  # Descrições dos programas
}

#Check se todos dados estão com o mesmo número de entradas
print(len(dados['Programa']),
len(dados['Descrição']),
len(dados['País_Moeda']),
len(dados['País do Programa']),
len(dados['Ano']),
len(dados['Ano2']),
len(dados['Gasto_Local']))

df = pd.DataFrame(dados)

117 117 117 117 117 117 117


## Conversor

In [7]:
# Aplicar a função para cada linha no DataFrame
df['Gasto_Convertido_Dolar_2024'] = df.apply(converter_para_dolar_corrente, axis=1)

# Aplicar Função para juntar Ano 1 e 2
df = definir_periodo(df)

# Aplicar as novas funções para as colunas 'Gasto_Local' e 'Gasto_Convertido_Dolar_2024'
df['Gasto Local'] = df.apply(lambda row: formatar_valor_moeda_local(row['Gasto_Local'], row['País_Moeda']), axis=1)
df['Gasto Convertido US$ 2024'] = df['Gasto_Convertido_Dolar_2024'].apply(formatar_valor_dolar)

# Exibir o DataFrame com as colunas formatadas
df[['País do Programa', 'Programa', 'Descrição', 'Ano', 'Ano2', 'Período', 'Gasto Local', 'Gasto Convertido US$ 2024']]

<ipython-input-4-ef2d360cadfc>:49: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return float(gasto_ajustado)


,País do Programa,Programa,Descrição,Ano,Ano2,Período,Gasto Local,Gasto Convertido US$ 2024
0,US,Defense Production Act (DPA),"Title I: Used to purchase over 187,000 ventila...",2020,0,2020,USD 2.90 bilhões,USD 3.50 bilhões
1,US,Defense Production Act (DPA),Title III: Increase capacity for nasal swabs a...,2020,0,2020,USD 208.00 milhões,USD 251.32 milhões
2,US,Defense Production Act (DPA),Obrigou este valor à compra de fornecimentos e...,2020,0,2020,USD 8.40 bilhões,USD 10.15 bilhões
3,US,Defense Production Act (DPA),Award for domestic production expansion projec...,2020,0,2020,USD 1.51 trilhões,USD 1.83 trilhões
4,US,Global Health Security,Experts estimate this additional amount annual...,2024,0,2024,USD 31.10 bilhões,USD 31.10 bilhões
...,...,...,...,...,...,...,...,...
112,IN,Programas de incentivos vinculados (PLI),Programa para medicamentos,2021,0,2021,INR 150.00 bilhões,USD 8.35 bilhões
113,IN,Programas de incentivos vinculados (PLI),Programa para dispositivos médicos,2020,0,2020,INR 34.20 bilhões,USD 2.03 bilhões
114,GB,Build Back Better: Our Plan for Growth and Inn...,Não específico para indústria da saúde: govern...,2021,0,2021,GBP 14.60 bilhões,USD 26.13 bilhões
115,GB,Build Back Better: Our Plan for Growth and Inn...,Não específico para indústria da saúde: fundin...,2021,0,2021,GBP 800.00 milhões,USD 1.43 bilhões


In [16]:
df_agrupado = agrupar_gastos(df)

# Convert 'Gasto_Convertido_Dolar_2024' to numeric before formatting
df_agrupado['Gasto_Convertido_Dolar_2024'] = pd.to_numeric(df_agrupado['Gasto_Convertido_Dolar_2024'], errors='coerce')

# Apply formatting only once
df_agrupado['Gasto_Convertido_Dolar_2024'] = df_agrupado['Gasto_Convertido_Dolar_2024'].apply(formatar_valor_dolar)

df_agrupado

,Período,País do Programa,Gasto_Convertido_Dolar_2024
0,2006,MX,USD 870.52 milhões
1,2007,MX,USD 921.28 milhões
2,2008,MX,USD 875.33 milhões
3,2008 à 2020,CN,USD 5.01 bilhões
4,2009,CN,USD 6.94
5,2009,MX,USD 330.72 milhões
6,2009 à 2016,CN,USD 2.21 trilhões
7,2011 à 2015,ZA,USD 3.51 milhões
8,2012,ZA,USD 426.87 milhões
9,2012 à 2017,IN,USD 90.03 milhões


In [15]:
df_pais = agrupar_gastos_pais(df)

# Convert 'Gasto_Convertido_Dolar_2024' to numeric before formatting
df_pais['Gasto_Convertido_Dolar_2024'] = pd.to_numeric(df_pais['Gasto_Convertido_Dolar_2024'], errors='coerce')

# Apply formatting only once
df_pais['Gasto_Convertido_Dolar_2024'] = df_pais['Gasto_Convertido_Dolar_2024'].apply(formatar_valor_dolar)

df_pais

,País do Programa,Gasto_Convertido_Dolar_2024
0,CN,USD 8.51 trilhões
1,GB,USD 1.94 trilhões
2,IN,USD 171.54 bilhões
3,KR,USD 497.76 bilhões
4,MX,USD 18.78 bilhões
5,US,USD 2.10 trilhões
6,ZA,USD 255.19 bilhões


In [20]:
df_prog = agrupar_gastos_programa(df)

# Convert 'Gasto_Convertido_Dolar_2024' to numeric before formatting
df_prog['Gasto_Convertido_Dolar_2024'] = pd.to_numeric(df_prog['Gasto_Convertido_Dolar_2024'], errors='coerce')

# Apply formatting only once
df_prog['Gasto_Convertido_Dolar_2024'] = df_prog['Gasto_Convertido_Dolar_2024'].apply(formatar_valor_dolar)

df_prog

,Programa,País do Programa,Gasto_Convertido_Dolar_2024
0,14º Plano Quinquenal Chinês (2021-2025),CN,USD 561.02 bilhões
1,14º Plano Quinquenal para o Desenvolvimento da...,CN,USD 5.01 bilhões
2,ARP A-H,US,USD 2.57 bilhões
3,ARPA-H,US,USD 7.50 bilhões
4,Advanced Manufacturing Plan,GB,USD 7.72 bilhões
5,American Pandemic Preparedness: Transforming O...,US,USD 75.37 bilhões
6,Bio-Health Industry Innovation Strategy,KR,USD 17.41 bilhões
7,Boston-Korea,KR,USD 77.46 milhões
8,Build Back Better: Our Plan for Growth and Inn...,GB,USD 27.56 bilhões
9,Cluster Development Programme for Pharma Secto...,IN,USD 90.24 milhões
